# Multi-image support

In [ ]:

import os
import numpy as np
import cupy as cp
from matplotlib import pyplot as plt


Setting resource folder

In [ ]:
resources_dir = os.getenv("PYNVIMGCODEC_EXAMPLES_RESOURCES_DIR", "../assets/images/")

Import nvImageCodec module and create Decoder

In [ ]:
from nvidia import nvimgcodec
decoder = nvimgcodec.Decoder()


Create CodeStream from tiff file

In [ ]:

img_full_path = os.path.abspath(os.path.join(resources_dir, "multi_page.tif"))

cs = nvimgcodec.CodeStream(img_full_path)

print(f"Number of tiles (Y): {cs.num_tiles_y}, Number of tiles (X): {cs.num_tiles_x}")
print(f"Tile height: {cs.tile_height}, Tile width: {cs.tile_width}")
print(f"Number of images: {cs.num_images}")
print(f"Image (0) height: {cs.height},  width: {cs.width}, number of channels: {cs.num_channels}, sample type: {cs.dtype}, size: {cs.size}")


Get sub code stream for first image (index 0) and decode it

In [ ]:
cs_0 = cs.get_sub_code_stream(0)

print(f"Number of tiles (Y): {cs_0.num_tiles_y}, Number of tiles (X): {cs_0.num_tiles_x}")
print(f"Tile height: {cs_0.tile_height}, Tile width: {cs_0.tile_width}")
print(f"Image (0) height: {cs_0.height}, width: {cs_0.width}, number of channels: {cs_0.num_channels}, sample type:{cs_0.dtype}, size: {cs_0.size}")

img_0 = decoder.decode(cs_0, params=nvimgcodec.DecodeParams(allow_any_depth=True, color_spec=nvimgcodec.ColorSpec.UNCHANGED))

plt.imshow(img_0.cpu())

Get sub code stream for image with index 2 and decode it

In [ ]:
cs_2 = cs.get_sub_code_stream(2)

print(f"Number of tiles (Y): {cs_2.num_tiles_y}, Number of tiles (X): {cs_2.num_tiles_x}")
print(f"Tile height: {cs_2.tile_height}, Tile width: {cs_2.tile_width}")
print(f"Image (2) height: {cs_2.height}, width: {cs_2.width}, number of channels: {cs_2.num_channels}, sample type:{cs_2.dtype}, size: {cs_2.size}")

img_2 = decoder.decode(cs_2, params=nvimgcodec.DecodeParams(allow_any_depth=True, color_spec=nvimgcodec.ColorSpec.UNCHANGED))

plt.imshow(img_2.cpu())

Get sub code streams for all images in the loop and decode them one by one

In [ ]:
for code_stream_idx in range(0, cs.num_images):
    sub_code_stream = cs.get_sub_code_stream(code_stream_idx)
    print (sub_code_stream.num_channels)
    print (sub_code_stream.sample_format)
    img = decoder.decode(sub_code_stream, params=nvimgcodec.DecodeParams(allow_any_depth=True, color_spec=nvimgcodec.ColorSpec.UNCHANGED))
    print(img.__cuda_array_interface__)
    print ("precision", img.precision)
    plt.figure()
    img_h = img.cpu() 

    if img.precision == 16:
       img_h =  (np.asarray(img_h)  / 65535.0 * 255).astype(np.uint8)
       
    plt.imshow(img_h, cmap=('gray' if img_h.shape[2] == 1 else None))

Get sub code streams for all images in the loop and decode them in batch

In [ ]:
sub_code_streams = []

for code_stream_idx in range(0, cs.num_images):
    sub_code_streams.append(cs.get_sub_code_stream(code_stream_idx))
    imgs = decoder.decode(sub_code_streams, params=nvimgcodec.DecodeParams(allow_any_depth=True, color_spec=nvimgcodec.ColorSpec.UNCHANGED))

for img in imgs:
    plt.figure()
    img_h = img.cpu() 

    if img.precision == 16:
       img_h =  (np.asarray(img_h)  / 65535.0 * 255).astype(np.uint8)
    plt.imshow(img_h, cmap=('gray' if img_h.shape[2] == 1 else None))

Decode region from particular image

In [ ]:
cs_6 = cs.get_sub_code_stream(5, nvimgcodec.Region(100, 175, 300, 400))

print(f"Number of tiles (Y): {cs_6.num_tiles_y}, Number of tiles (X): {cs_6.num_tiles_x}")
print(f"Tile height: {cs_6.tile_height}, Tile width: {cs_6.tile_width}")
print(f"Image (8) height: {cs_6.height}, width: {cs_6.width}, number of channels: {cs_6.num_channels}, sample type: {cs_6.dtype}, size: {cs_6.size}")

img_6 = decoder.decode(cs_6, params=nvimgcodec.DecodeParams(allow_any_depth=True, color_spec=nvimgcodec.ColorSpec.UNCHANGED))

img_h = img_6.cpu() 

if img_6.precision == 16:
    img_h =  (np.asarray(img_h)  / 65535.0 * 255).astype(np.uint8)
plt.imshow(img_h, cmap=('gray' if img_h.shape[2] == 1 else None))
